<a href="https://colab.research.google.com/github/Ali-hassan-yousaf/final_year_project/blob/main/lost_found.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio faiss-cpu torch torchvision Pillow numpy

import gradio as gr
import numpy as np
import os
import faiss
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import resnet50, ResNet50_Weights
import datetime

# Initialize model for feature extraction
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model = model.to(device).eval()

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class PetFinderAPI:
    def __init__(self):
        self.index = faiss.IndexFlatL2(2048)  # ResNet50 feature dimension
        self.image_data = []
        self.image_dir = "pet_images"
        os.makedirs(self.image_dir, exist_ok=True)

    def extract_features(self, image_np):
        img = Image.fromarray(image_np).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            features = model(img_tensor)
        return features.cpu().numpy().flatten()

    def add_found_pet(self, image_np, filename, found_location, pet_type, phone_number, notes):
        try:
            # Save image
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            save_filename = f"{timestamp}_{filename}"
            save_path = os.path.join(self.image_dir, save_filename)
            Image.fromarray(image_np).save(save_path)

            # Extract and store features
            features = self.extract_features(image_np)
            self.image_data.append({
                "image": image_np,
                "filename": save_filename,
                "features": features,
                "found_location": found_location,
                "pet_type": pet_type,
                "phone_number": phone_number,
                "notes": notes,
                "timestamp": timestamp
            })

            # Update FAISS index
            if len(self.image_data) == 1:
                self.index.add(np.array([features]))
            else:
                self.index.add(np.array([features]))

            return save_path, f"✅ Pet added successfully!\nSaved as: {save_filename}"
        except Exception as e:
            return None, f"❌ Error: {str(e)}"

    def search_lost_pet(self, query_image_np):
        try:
            if not self.image_data:
                return [], "⚠️ Database is empty. Please add pets first."

            # Extract query features
            query_features = self.extract_features(query_image_np)

            # Search in FAISS index
            distances, indices = self.index.search(np.array([query_features]), 3)

            # Prepare results
            results = []
            for idx, score in zip(indices[0], distances[0]):
                if idx >= 0:  # FAISS returns -1 for invalid indices
                    result = self.image_data[idx].copy()
                    result['score'] = 1 / (1 + score)  # Convert distance to similarity score
                    results.append(result)

            if not results:
                return results, "🔍 No matches found. Try another image."

            return results, f"🔎 Found {len(results)} potential matches!"
        except Exception as e:
            return [], f"❌ Search error: {str(e)}"

# Initialize API
pet_api = PetFinderAPI()

# Gradio Functions
def add_found_pet_gr(image, found_location, pet_type, phone_number, notes):
    image_np = np.array(image)
    save_path, msg = pet_api.add_found_pet(
        image_np,
        "uploaded.jpg",
        found_location,
        pet_type,
        phone_number,
        notes
    )
    return msg

def search_lost_pet_gr(query_image):
    query_np = np.array(query_image)
    results, msg = pet_api.search_lost_pet(query_np)

    # Prepare outputs
    images = [None, None, None]
    details = ["", "", ""]

    if results:
        for i, r in enumerate(results[:3]):
            images[i] = Image.fromarray(r['image'])
            details[i] = (
                f"🐾 Type: {r.get('pet_type', 'N/A')}\n"
                f"📍 Location: {r.get('found_location', 'N/A')}\n"
                f"📞 Contact: {r.get('phone_number', 'N/A')}\n"
                f"📝 Notes: {r.get('notes', 'N/A')}\n"
                f"⏰ Reported: {r.get('timestamp', 'N/A')}"
            )

    return msg, images[0], images[1], images[2], details[0], details[1], details[2]

# Custom Theme with #5e5eee Colored Buttons
theme = gr.themes.Soft(
    primary_hue="indigo",
    secondary_hue="gray",
    font=[gr.themes.GoogleFont("Montserrat")]
).set(
    button_primary_background_fill="#5e5eee",       # Main button color
    button_primary_background_fill_hover="#4a4ae0",  # Slightly darker on hover
    button_primary_text_color="#ffffff",             # White text
    button_primary_background_fill_dark="#5e5eee",   # Same for dark mode
    button_primary_border_color="#5e5eee",
    button_primary_border_color_dark="#5e5eee",
    button_primary_border_color_hover="#4a4ae0"
)

# Create Interface
with gr.Blocks(theme=theme, title="Pet Finder") as demo:
    gr.Markdown("# 🐾 Lost & Found")


    with gr.Tab("🔍 Search Lost Pet", id="search_tab"):
        with gr.Row():
            with gr.Column():
                search_input = gr.Image(type="pil", label="Upload photo of lost pet", sources=["upload", "webcam"])
                search_btn = gr.Button("Search Database", variant="primary")
            with gr.Column():
                search_output_text = gr.Textbox(label="Results", interactive=False)
                with gr.Row():
                    with gr.Column():
                        search_output1 = gr.Image(label="Top Match", show_label=True)
                        # Added scrollable text area with fixed height
                        search_detail1 = gr.Textbox(label="Details", interactive=False, show_label=True, lines=5, max_lines=10)
                    with gr.Column():
                        search_output2 = gr.Image(label="Second Match", show_label=True)
                        # Added scrollable text area with fixed height
                        search_detail2 = gr.Textbox(label="Details", interactive=False, show_label=True, lines=5, max_lines=10)
                    with gr.Column():
                        search_output3 = gr.Image(label="Third Match", show_label=True)
                        # Added scrollable text area with fixed height
                        search_detail3 = gr.Textbox(label="Details", interactive=False, show_label=True, lines=5, max_lines=10)

        search_btn.click(
            search_lost_pet_gr,
            inputs=search_input,
            outputs=[
                search_output_text,
                search_output1,
                search_output2,
                search_output3,
                search_detail1,
                search_detail2,
                search_detail3
            ]
        )

    with gr.Tab("📸 Add Found Pet", id="add_tab"):
        with gr.Row():
            with gr.Column():
                add_input = gr.Image(type="pil", label="Upload photo of found pet", sources=["upload", "webcam"])
                found_location = gr.Textbox(label="Found Location", placeholder="Where was the pet found?")
                with gr.Row():
                    pet_type = gr.Dropdown(
                        ["Dog", "Cat", "Bird", "Rabbit", "Other"],
                        label="Pet Type",
                        value="Dog"
                    )
                    phone_number = gr.Textbox(label="Contact Phone", placeholder="Your phone number")
                # Added scrollable text area for notes with fixed height
                notes = gr.Textbox(label="Additional Notes", placeholder="Color, collar, condition, etc.", lines=3)
                add_btn = gr.Button("Add to Database", variant="primary")
            with gr.Column():
                add_output = gr.Textbox(label="Status", interactive=False)

        add_btn.click(
            add_found_pet_gr,
            inputs=[add_input, found_location, pet_type, phone_number, notes],
            outputs=add_output
        )

    gr.Markdown("---\n> ℹ️ Tips: Use clear, well-lit photos for best results")

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.3 MB/s eta 0:00:00


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 145MB/s]


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5d0557481cec0606c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
wait